# Topic Modeling (LDA)
## Setup


In [1]:
!pip install --upgrade gensim

In [2]:
!pip install nltk
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.1 MB/s eta 0:00:00


## import libraries

In [3]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import numpy as np
import json
import glob

#gensim
import gensim
import gensim.corpora as copora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Data Prep

In [5]:
def laod_data(file):
  with open (file, "r", encoding="utf-8") as f:
    data = json.load(f)
  return (data)

def write_data(file, data):
  with open (file, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4)

In [6]:
stopwords = stopwords.words("english")
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [24]:
data = laod_data("/content/Extraversion_answers_and_comments.json")["answers"]

print(data[0][0:90])

How to make a great R reproducible example


In [25]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
  nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
  texts_out = []
  for text in texts:
    doc = nlp(text)
    new_text = []
    for token in doc:
      if token.pos_ in allowed_postags:
        new_text.append(token.lemma_)
    final = " ".join(new_text)
    texts_out.append(final)
  return(texts_out)

lemmatized_text = lemmatization(data)
print(lemmatized_text[0][0:90])

make great r reproducible example


In [26]:
def gen_words(texts):
  final = []
  for text in texts:
    new = gensim.utils.simple_preprocess(text, deacc=True)
    final.append(new)
  return (final)

data_words = gen_words(lemmatized_text)

print(data_words[0][0:30])

['make', 'great', 'reproducible', 'example']


In [27]:
id2word = gensim.corpora.Dictionary(data_words)

corpus = []

for text in data_words:
  new = id2word.doc2bow(text) # bow -> bag of words
  corpus.append(new)

print(corpus[0][0:20]) #each tuple contains index of the word along with the frequency of that word.

[(0, 1), (1, 1), (2, 1), (3, 1)]


In [28]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                    id2word=id2word,
                                    num_topics=10,
                                    random_state=100,
                                    update_every=1,
                                    chunksize=100,
                                    passes=10,
                                    alpha="auto")

## Visualization of Data

In [29]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word, mds="mmds", R=9)
vis

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.384597  0.260066       1        1  15.015124
8      0.117280  0.432208       2        1  12.151284
7     -0.320938 -0.232350       3        1  12.002433
1     -0.255246  0.419370       4        1  11.600927
2      0.198539 -0.435395       5        1  11.028902
9     -0.173454  0.161726       6        1  10.412217
4     -0.123424 -0.434783       7        1   7.877360
3     -0.483192  0.014062       8        1   7.487638
6      0.185468 -0.089984       9        1   6.212719
5      0.470371 -0.094920      10        1   6.211397, topic_info=         Term         Freq        Total Category  logprob  loglift
274       use  6370.000000  6370.000000  Default   9.0000   9.0000
465      file  3048.000000  3048.000000  Default   8.0000   8.0000
29   function  2467.000000  2467.000000  Default   7.0000   7.0000
147     check  1982.000000  1982.000000  Default   6.0000   6.0000
63       work  1928.000000  1928.000000  Default   5.0000   5.0000
..        ...          ...          ...      ...      ...      ...
353     table   969.797902   970.741735  Topic10  -2.9786   2.7778
190     valid   785.521928   786.465679  Topic10  -3.1893   2.7776
324     image   723.965698   724.909499  Topic10  -3.2709   2.7775
188      same   687.409560   688.353338  Topic10  -3.3228   2.7774
511   default   483.368758   484.312825  Topic10  -3.6749   2.7768

[104 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
290       3  0.998444        act
625       2  0.998711      array
191       5  0.999048     become
87        8  0.999185       bike
1133      1  0.999342  bootstrap
...     ...       ...        ...
28        4  0.999012      value
279       1  0.998821   variable
131       7  0.998290       very
280       1  0.998256        way
63        3  0.999752       work

[98 rows x 3 columns], R=9, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 9, 8, 2, 3, 10, 5, 4, 7, 6])

In [30]:
print('\nPerplexity: ', lda.log_perplexity(corpus,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.733227404433795

Coherence Score:  0.5103639515249616
